<div>
<br><br><br><br><br><br><br><br><br><br>
<br><br><br><br><br>
</div>

<div align="center">
<h1>Springboard DSC Capstone Project 2</h1>

<h3>Google Analytics Customer Revenue Prediction</h3>
<h3>Neha Jain Surana</h3>
<h3>Nov 2018</h3>
</div>

<div>
<br><br><br><br><br><br><br><br><br><br>
<br><br><br><br><br><br><br><br><br><br>
</div>

[Introduction](#section1)

[Client](#section2)

[Data and its acquisition](#section3)

[Data Wrangling](#section4)
- [Parse Json columns](#section5)
- [Delete non useful columns](#section6)
- [Fill up missing values](#section7)
- [Check validity of geo related data](#section8)

[Data Exploration](#section5)
- [Is there class imbalance?](#section6)
- [Visualizing the data](#section7)
- [Are some features related to each other?](#section8)
- [Are the correlations statistically significant?](#section9)
- [Important features to predict the target product category](#section10)
- [Features dimension reduction](#section11)
    - [PCA - Principal Component Analysis](#section12)
    - [T-SNE (T-Distributed Stochastic Neighbouring Entities) visualiztion](#section13)
    - [LDA - Linear Discriminant Analysis](#section14)
    
[Data Modeling](#section15)

- [Theory and Approach followed](#section16)
- [Overfitting and Underfitting](#section17)
- [Model evaluation criteria](#section18)
- [Data and code organization](#section19)

- [First (Baseline) Model](#section20)
    - [Baseline Results](#section21)
    - [Held out test set](#section22)
    - [Logistic Regression](#section23)
    
    
- [Dealing with class imbalance](#section24)
    - [Over sampling](#section25)
    - [Under sampling](#section26)
    - [Combination of oversampling and undersampling](#section27)
    - [Balancing using weights](#section28)
    
    
- [Other simple Models](#section29)
    - [MultinomialNB](#section30)
    - [Support Vector Machines](#section31)
    - [KNN](#section32)
    
- [Ensembling and Stacking Models](#section33)
    - [Bagging Models - Random Forest](#section34)        
    - [Boosting Models - Gradient Boosting](#section35)
    - [Boosting Models - XGB](#section36)
- [Stacking Models](#section37)

[Model Comparisons](#section38)

[Recommendations for the client](#section39)

[Conclusions and Future Analysis](#section40)


<a id='section1'></a>
# Introduction

The primary purpose of any business is to make profits for its owners and stakeholders. Also, the 80/20 rule has proven true for many businesses–only a small percentage of customers produce most of the revenue. 

For an e-store like Google merchandise store, it is nothing different. As such, marketing teams are challenged to make appropriate investments in promotional strategies. But for that, they need as accurate a prediction as possible, of the revenue per customer.

Although, the main prediction is not for future, i.e. we predict revenues for the visits already done. But there might be a second phase of the project where we predict for future dates.

<a id='section2'></a>
# Client

Here the client is the company Google and its specific e-store called Google Merchandise Store (GStore). We hope to provide highly accurate revenue predictions per customer visiting the GStore. The outcome should be more actionable operational changes and a better use of marketing budgets. Also, it would be serve as an evidence for those companies who choose to use data analysis on top of GA data, because the current data comes from the same.

<a id='section3'></a>
# Data and its acquisition

The data is provided by the client in the form of csv file. There is some additional test data that can be used to evaluate and report the algorithm success. The main data file has around 900 thousand rows corresponding to each cusomter visit to the GStore. There are multiple columns which contain JSON blobs of varying depth. In one of those JSON columns, totals, the sub-column transactionRevenue contains the revenue information we are trying to predict.

Loading the data will not be very simple because the json blobs need to be parsed. I am using the read_csv function provided by the python-pandas library to load the data into datafrme. Json converters are used to parse the Json fields. The result is a table like structure that can be used for processing.

Some of the Data Fields:
- <b>fullVisitorId</b> - A unique identifier for each user of the Google Merchandise Store.
- <b>channelGrouping</b> - The channel via which the user came to the Store.
- <b>date</b> - The date on which the user visited the Store.
- <b>device</b> - The specifications for the device used to access the Store.
- <b>geoNetwork</b> - This section contains information about the geography of the user.
- <b>sessionId</b> - A unique identifier for this visit to the store.
- <b>socialEngagementType</b> - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
- <b>totals</b> - This section contains aggregate values across the session.
- <b>trafficSource</b> - This section contains information about the Traffic Source from which the session originated.
- <b>visitId</b> - An identifier for this session. This is part of the value usually stored as the _utmb_ cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.
- <b>visitNumber</b> - The session number for this user. If this is the first session, then this is set to 1.
- <b>visitStartTime</b> - The timestamp (expressed as POSIX time).
- <b>hits</b> - This row and nested fields are populated for any and all types of hits. Provides a record of all page visits.
- <b>customDimensions</b> - This section contains any user-level or session-level custom dimensions that are set for a session. This is a repeated field and has an entry for each dimension that is set.
- <b>totals</b> - This set of columns mostly includes high-level aggregate data.


<a id='section4'></a>
# Data Wrangling
Metadata about number of unique and null values for each column, helps to decide what clean up and preprocessing of the data is needed. 

<a id='section5'></a>
### Parse Json columns
Columns ('device', 'geoNetwork', 'totals', 'trafficSource') were found to be json blobs. Parsed these columns and added them back to the train/test data.
Train data has 55 columns and 903653 rows. The data types are: bool(1), int64(4), object(50)


<a id='section6'></a>
### Delete non useful columns
Columns with 1 unique value and zero null value are not useful in any way and we delete them.

<img src="../images/non-useful-cols.png" width="400" align="left">

<a id='section7'></a>
### Fill up missing values
The columns having only one unique value other than nulls, can be treated as boolean. The null can be treated as True or False depending on the other value present. But from the perspective of fitting the model it doesn't matter.

    Column                                   : Old values            : New values
    totals.bounces                           : ['1' nan]             : [1 0]
    totals.newVisits                         : ['1' nan]             : [1 0]
    trafficSource.adwordsClickInfo.isVideoAd : [nan False]           : [1 0]
    trafficSource.campaignCode               : [nan '11251kjhkvahf'] : [1 0]
    trafficSource.isTrueDirect               : [nan True]            : [0 1]

The columns having multiple values and nulls, have to filled up for missing values.

    Column                                       : Unique values  : Null values 
    
    totals.pageviews                             : 213            : 100         
    (Very few null values. null values might indicate no page view. so fill up missing by '0's.)
    
    totals.transactionRevenue                    : 5332                  : 892138
    (Huge number of null values. Null revenue means '0', so replace by it.)
    
    trafficSource.adwordsClickInfo.page          : 8                     : 882193
    (Huge number of null values. So might not be important variable. It's an int, so convert it to int data type. Filling up null with '0's.)
    
    trafficSource.adContent                      : 44                    : 892707
    trafficSource.adwordsClickInfo.adNetworkType : 2                     : 882193
    trafficSource.adwordsClickInfo.slot          : 2                     : 882193
    trafficSource.keyword                        : 3659                  : 502929
    trafficSource.referralPath                   : 1475                  : 572712
    (Huge number of null values. So might not be important variable. It's a string value. Filling up nulls by 'Unknown')
    
    trafficSource.adwordsClickInfo.gclId         : 17774                 : 882092
    (It's alpha numeric strings. Filling up nulls by 'Unknown')        


<a id='section8'></a>
### Check validity of geo related data and clean up if needed
- Find out unique geo combinations. 
- There are values like '(not set)' and 'not available in demo dataset'. We can't do much about them.
- Divided the 6 geo columns into 2 parts. The unique combinations of country, sub-continent, continent seems to be fine. There are 222 such rows. Eyeballing them also looks good.
- The other unique combinations for city, metro, region seems to have some invalid data. Same city is associated with different regions. This can happen when same city name occurs in two places. So we further valide such combinations.
- We found that many of such combinations don't have valid match with country. We delete 3765 such rows. For e.g. 

<img src="../images/invalid-geo-combs.png" width="800" align="left">


<a id='section9'></a>
# Data Exploration

Let us explore all the fields and their relation to the target variable.

<a id='section10'></a>
### Analyze the target variable

The given transaction revenues data is actually the original revenue multiplied by a million. So actual revenues are millionth of the given numbers. The reason behind is to keep the precision of the numbes upto certain length, so that the values are not rounded while being transformed in multiple systems.

The percent of revenue generating sessions is only 1.27%. And that means most of the visits don't generate any revenue.

<img src="../images/non-zero-rev-visits.png" width="800" align="left">

<img src="../images/non-zero-rev-visits-cumul.png" width="800" align="left">


Few things noticed by comparing revenue generating visits vs no-revenue visits. So these all could serve as good features as they have differences in values for revenue vs no-revenue visits.
- Bounces are all zeros for revenue generating visits
- Campaign code are all ones
- Maximum visitNumber is 315 vs 395
- Minimum pageview is 2 vs 0
- Maximum pageviews is 469 vs 429
- Maximum adwordclickinfo page is 1 vs 14



<a id='section11'></a>
### Analyze the visitorIds

There are 714167 unique visitor ids for the total of 903653 visits. This tells that there will be huge number of visitors who visit the store only once.

Percentage of unique users for:
- revenue visits 88.25992150021806
- non revenue visits 80.88129190712083
- all visits 80.25376491296694
    
Percent for unique visitors is slightly more for revenue visits as compared to non revenue visits which is little weird because we would expect more visits by some visitor when they buy something.

<img src="../images/visits-rev.png" width="800" align="left">

<img src="../images/visits-tot-rev.png" width="800" align="left">

<img src="../images/visits-perct-rev.png" width="800" align="left">

Above all analysis means that when the number of visits by a visitor increases it necessarily doesn't mean that more revenue will be generated

<img src="../images/session-rev.png" width="800" align="left">

Understanding about visitorIds and revenues
- Revenues are evenly spread for all the visits
- Proportion of revenue visits decreases as the number of visits by the visitor increases. That is, if a visitor visits the site multiple times, it doesn't mean he/she is buying more and more.
- Total revenue increases slights if the number of visits increase
- Some visitors have generated revenue multiple times in multiple visits. Maximum is 30 revenue transactions by a visitor.

<a id='section12'></a>
### Analyze the date and visitStartTime

We parse the Date and VisitStartTime into following components and visualize them with respect to the revenue. Also, show below are some interesting charts.
- Year
- Month
- Day (of the month)
- Day (of the year, Jan 1 is day 1)
- Weekday
- Hour
- Local hour (local hour based on timezone)

<img src="../images/month-rev.png" width="800" align="left">
<img src="../images/day-month-rev.png" width="800" align="left">
<img src="../images/weekday-rev.png" width="800" align="left">
<img src="../images/hour-rev.png" width="800" align="left">
<img src="../images/local-hour-rev.png" width="800" align="left">
<img src="../images/visit-start-rev.png" width="800" align="left">
<img src="../images/day-session.png" width="800" align="left">
<img src="../images/day-all.png" width="800" align="left">


Understanding about date and visitStartTime
- The visit counts increase in the festival months. But the mean revenue fluctuates in those months.
- Both the visit counts and mean revenues drop on weekends. May be people are busy doing other stuff then buying things online.
- Assuming large portion of visitors are from North America, the 24 hour pattern shows mean revenue increase in day time, maximizes around 8pm and then drops.
- We dont see any pattern for day of the month mean revenue. And the visit counts are pretty uniform. It means all days of the month are same as far as visiting the GStore is considered.
- We see weekly patterns for visits and therefore mean and total revenues. And although the visits surge around 50-120 days the mean revenues surge around 120-150 days.

<a id='section13'></a>
### Analyze visitNumber - visit number of a visitor

Understanding about visitNumber
- Most of the visitors visit less than 5 times. For e.g. number of visit first or second time is around 1 million and it drops very fast. Very few visitors visit for more than 50th time or more.

- Same is the case for visits that produce revenue.

- It is little strange that there are very few revenue visits such that it is 150-250th visit for that visitor.

- Mean revenues are uniformly distributed from 0.1 to 10 for visit numbers 1 to 150. And it is significantly higher for greater visit numbers. It indicates that oftern more number of visits are needed before high valued purchase.

<img src="../images/visit-no-rev.png" width="800" align="left">

<a id='section14'></a>
### Analyze channelGrouping

The counts and mean revenues for different channels are almost inversely proportional. That is, the mean revenue is high if a channel grouping is less frequent. For e.g. 'Display' channel.

<img src="../images/channel.png" width="800" align="left">

<a id='section15'></a>
### Analyze device fields

<img src="../images/browser.png" width="1000" align="left">

<img src="../images/device-category.png" width="800" align="left">

<img src="../images/is-mobile.png" width="800" align="left">

<img src="../images/os.png" width="800" align="left">

<a id='section16'></a>
### Analyze Geo fields

<img src="../images/continent.png" width="800" align="left">
<img src="../images/subcontinent.png" width="1000" align="left">
<img src="../images/country-visit.png" width="800" align="left">
<img src="../images/country-rev.png" width="800" align="left">
<img src="../images/city-all.png" width="1000" align="left">


<a id='section13'></a>
### Analyze new visits

More than 70% visits are new visits or first time visits. But as expected the mean revenue from old visitors is high as compared to new visitors.

<img src="../images/new-visits.png" width="800" align="left">

<a id='section13'></a>
### Analyze hits

Scatter plot indicates that hits below 100 get huge number of transactions as compared to other hit values. Page views has similar distribution. And it shows that hits and page views are linearly related.

<img src="../images/hits.png" width="800" align="left">
<img src="../images/hits-scatter.png" width="800" align="left">